In [2]:
from models import DualReadoutMACE
from Calculator import DeltaMaceCalculator

import torch
import torch.nn as nn
from typing import Dict

/home/krystynasyty/myenv/lib/python3.12/site-packages/e3nn/o3/_wigner.py:10: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  _Jd, _W3j_flat, _W3j_indices = torch.load(os.path.join(os.path.dirname(__file__), 'constants.pt'))


cuequivariance or cuequivariance_torch is not available. Cuequivariance acceleration will be disabled.


In [4]:
if __name__ == '__main__':
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # --- 1. Wczytaj modele ---
    MODEL_PATH = './MACE-MP_small.model'
    print(f"Wczytywanie modelu bazowego z: {MODEL_PATH}")
    base_model = torch.load(MODEL_PATH, map_location=device, weights_only=False)
    dual_model = DualReadoutMACE(base_model, mlp_hidden_features=[8], mlp_activation = "swiglu")

    # --- 2. Wydrukuj podsumowanie finalnego modelu ---
    print("\n" + "="*60)
    print("      Podsumowanie finalnego modelu 'DualReadoutMACE'")
    print("="*60)
    print(dual_model)
    print("-"*60)

    # --- 3. Wydrukuj szczegóły dotyczące parametrów trenowalnych vs zamrożonych ---
    total_params = sum(p.numel() for p in dual_model.parameters())
    trainable_params = sum(p.numel() for p in dual_model.parameters() if p.requires_grad)

    print(f"\nCałkowita liczba parametrów: {total_params:,}")
    print(f"Liczba trenowalnych parametrów: {trainable_params:,}")
    print("\nSzczegóły parametrów trenowalnych:")
    found_trainable = False
    for name, param in dual_model.named_parameters():
        if param.requires_grad:
            print(f"  - Warstwa: '{name}' | Rozmiar: {param.shape} | Status: Trenowalna")
            found_trainable = True
    
    if not found_trainable:
        print("  - Brak trenowalnych parametrów.")
        
    print("="*60)



Wczytywanie modelu bazowego z: ./MACE-MP_small.model
Freezing parameters of the entire base MACE model...
Detected 128 input features for the readout head.
Initializing non-linear MLP readout with activation 'swiglu' and hidden features: [8]
Weights and bias of the final layer of 'delta_readout' MLP have been initialized to zero.
No vacuum energy shifts provided; model will predict the residual delta directly.
Hook correctly registered on the last 'product' block.

      Podsumowanie finalnego modelu 'DualReadoutMACE'
DualReadoutMACE(
  (mace_model): ScaleShiftMACE(
    (node_embedding): LinearNodeEmbeddingBlock(
      (linear): Linear(89x0e -> 128x0e | 11392 weights)
    )
    (radial_embedding): RadialEmbeddingBlock(
      (bessel_fn): BesselBasis(r_max=6.0, num_basis=10, trainable=False)
      (cutoff_fn): PolynomialCutoff(p=5.0, r_max=6.0)
    )
    (spherical_harmonics): SphericalHarmonics()
    (atomic_energies_fn): AtomicEnergiesBlock(energies=[[-3.6672, -1.3321, -3.4821, -4.736

In [4]:
import ase.build
from mace.calculators import MACECalculator
from ase.io import read, write


MODEL_FILE = "MACE-MP_small.model"

MP_calc = MACECalculator(model_paths=MODEL_FILE)


calculator = DeltaMaceCalculator(model=dual_model, device="cpu")


atoms = read('/home/krystynasyty/X23/08_cyanamide/solid.xyz')



Using head Default out of ['Default']
No dtype selected, switching to float64 to match model dtype.
Using a provided DualReadoutMACE model object.
Calculator initialized on device 'cpu' with dtype 'float64'.


/home/krystynasyty/myenv/lib/python3.12/site-packages/mace/calculators/mace.py:197: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


In [5]:
print("Energie i siły dla C6H6 z Dual Macem")
atoms.calc = calculator
print("Energy: ", atoms.get_potential_energy())
print("Forces: ", atoms.get_forces())

Energie i siły dla C6H6 z Dual Macem
Energy:  -261.7232793098623
Forces:  [[-0.13207055 -0.21598913  0.03055513]
 [ 0.1320688   0.21595558 -0.03049507]
 [ 0.13151725  0.21669984  0.03252289]
 [-0.1315023  -0.21676863 -0.03253388]
 [ 0.13182563 -0.21727859 -0.03365709]
 [-0.13183872  0.21725984  0.03365605]
 [-0.13269699  0.21639711 -0.03225566]
 [ 0.1326926  -0.2165173   0.03245291]
 [ 0.06844613 -0.48608869  0.76075774]
 [-0.06843661  0.48600949 -0.7605854 ]
 [-0.06855374  0.48969539  0.76808913]
 [ 0.06853961 -0.4896363  -0.76802157]
 [-0.06866279 -0.49073558 -0.77051404]
 [ 0.06866926  0.49071363  0.77048039]
 [ 0.06841559  0.48799178 -0.76429364]
 [-0.06849969 -0.48751959  0.76356104]
 [ 0.02392374  0.43543527 -0.61546006]
 [-0.02392608 -0.4353465   0.61525033]
 [-0.02374258 -0.4388066  -0.62259698]
 [ 0.02374679  0.43880389  0.62255863]
 [-0.02353615  0.44132914  0.62777213]
 [ 0.02353123 -0.44129826 -0.62771168]
 [ 0.02408374 -0.43836749  0.62112219]
 [-0.02393915  0.43820703 -0.

In [6]:
print("Test z zerowymi wagami warsty dela learninowej")
print("Energie i siły dla C6H6 z MP-0 small")
atoms.calc = MP_calc
print("Energy: ", atoms.get_potential_energy())
print("Forces: ", atoms.get_forces())

Test z zerowymi wagami warsty dela learninowej
Energie i siły dla C6H6 z MP-0 small
Energy:  -261.7232793098623
Forces:  [[-0.13207055 -0.21598913  0.03055513]
 [ 0.1320688   0.21595558 -0.03049507]
 [ 0.13151725  0.21669984  0.03252289]
 [-0.1315023  -0.21676863 -0.03253388]
 [ 0.13182563 -0.21727859 -0.03365709]
 [-0.13183872  0.21725984  0.03365605]
 [-0.13269699  0.21639711 -0.03225566]
 [ 0.1326926  -0.2165173   0.03245291]
 [ 0.06844613 -0.48608869  0.76075774]
 [-0.06843661  0.48600949 -0.7605854 ]
 [-0.06855374  0.48969539  0.76808913]
 [ 0.06853961 -0.4896363  -0.76802157]
 [-0.06866279 -0.49073558 -0.77051404]
 [ 0.06866926  0.49071363  0.77048039]
 [ 0.06841559  0.48799178 -0.76429364]
 [-0.06849969 -0.48751959  0.76356104]
 [ 0.02392374  0.43543527 -0.61546006]
 [-0.02392608 -0.4353465   0.61525033]
 [-0.02374258 -0.4388066  -0.62259698]
 [ 0.02374679  0.43880389  0.62255863]
 [-0.02353615  0.44132914  0.62777213]
 [ 0.02353123 -0.44129826 -0.62771168]
 [ 0.02408374 -0.4383

In [11]:
from ase import Atoms
import numpy as np

print("____Testy niezmienniczości energii ze względu na permutajcę atomów_____")

molecule = Atoms('H2O', positions=[(0.1, 0.2, 0.3), (0.9, -0.1, 0.2), (-0.5, 0.4, -0.6)])

molecule.calc = calculator

try:
    original_energy = molecule.get_potential_energy()
    print(f"Energia początkowa: {original_energy:.6f} eV")
except Exception as e:
    print(f"Błąd podczas obliczania energii początkowej: {e}")

        
print("-" * 30)

print("Test niezmienniczości permutacyjnej...")
permuted_molecule = molecule.copy()

indices = list(range(len(permuted_molecule)))
indices[0], indices[1] = indices[1], indices[0]
permuted_molecule = permuted_molecule[indices]

permuted_molecule.calc = calculator
permuted_energy = permuted_molecule.get_potential_energy()

print(f"Energia po permutacji atomów H: {permuted_energy:.6f} eV")

# Sprawdzenie, czy energie są takie same (z małą tolerancją numeryczną)
if np.allclose(original_energy, permuted_energy):
    print("✅ Test permutacji ZDANY")
else:
    print("❌ Test permutacji NIEZDANY")
        
print("-" * 30)

print("Test niezmienniczości rotacyjnej...")
rotated_molecule = molecule.copy()

    # Obróć cząsteczkę o dowolne kąty, np. 33 stopnie wokół osi 'x' i 58 wokół 'y'
rotated_molecule.rotate(33, 'x', center='COM')
rotated_molecule.rotate(58, 'y', center='COM')

    # Upewnij się, że kalkulator jest przypisany do obróconej cząsteczki
rotated_molecule.calc = calculator
rotated_energy = rotated_molecule.get_potential_energy()

print(f"Energia po rotacji: {rotated_energy:.6f} eV")

    # Sprawdzenie, czy energie są takie same
if np.allclose(original_energy, rotated_energy):
    print("✅ Test rotacji ZDANY")
else:
    print("❌ Test rotacji NIEZDANY")
        
print("-" * 30)


____Testy niezmienniczości energii ze względu na permutajcę atomów_____
Energia początkowa: -9.439025 eV
------------------------------
Test niezmienniczości permutacyjnej...
Energia po permutacji atomów H: -9.439025 eV
✅ Test permutacji ZDANY
------------------------------
Test niezmienniczości rotacyjnej...
Energia po rotacji: -9.439025 eV
✅ Test rotacji ZDANY
------------------------------


In [13]:
from ase import Atoms
import numpy as np

print("____Testy niezmienniczości energii ze względu na permutajcę atomów_____")

molecule =read('/home/krystynasyty/X23/08_cyanamide/solid.xyz')

molecule.calc = calculator

try:
    original_energy = molecule.get_potential_energy()
    print(f"Energia początkowa: {original_energy:.6f} eV")
except Exception as e:
    print(f"Błąd podczas obliczania energii początkowej: {e}")

        
print("-" * 30)

print("Test niezmienniczości permutacyjnej...")
permuted_molecule = molecule.copy()

indices = list(range(len(permuted_molecule)))
indices[0], indices[1] = indices[1], indices[0]
permuted_molecule = permuted_molecule[indices]

permuted_molecule.calc = calculator
permuted_energy = permuted_molecule.get_potential_energy()

print(f"Energia po permutacji atomów H: {permuted_energy:.6f} eV")

# Sprawdzenie, czy energie są takie same (z małą tolerancją numeryczną)
if np.allclose(original_energy, permuted_energy):
    print("✅ Test permutacji ZDANY")
else:
    print("❌ Test permutacji NIEZDANY")
        
print("-" * 30)



____Testy niezmienniczości energii ze względu na permutajcę atomów_____
Energia początkowa: -261.723279 eV
------------------------------
Test niezmienniczości permutacyjnej...
Energia po permutacji atomów H: -261.723279 eV
✅ Test permutacji ZDANY
------------------------------


In [ ]:
sprawdzić dysocjację molekół